In [3]:
import os
from __init__ import *

__ORIG_WD__ = os.getcwd()
os.chdir(f"{__ORIG_WD__}/../data_collectors")

from covid19_genome import Covid19Genome

os.chdir(__ORIG_WD__)


In [4]:
dc = Covid19Genome()

Building Data frame
Done building Data frame
Building remote dicts
Done building remote dicts
Building local dicts
Done building local dicts


In [ ]:
lineages = dc.getLocalLineages()

mappings = []
accessions = for lineage in lineages:
    for accession in dc.getLocalAccessionsPaths(lineage):
        mappings.append((accession, lineage))
